In [43]:



import os
import sys
import random
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import ScalarFormatter


# 获取输入和输出文件名
# FILE_NAME = '../data/trace/iperf_pacing_single_flow.csv'
# OUT_FILE_NAME = '../data/csv/iperf_pacing_single_flow.csv'

temp = '/home/hzheng/2023/wave/data/zzc/zzc_packets10'
FILE_NAME = temp + '.csv'
OUT_FILE_NAME = temp + '_wid.csv'

FILE_TYPE = os.path.splitext(FILE_NAME)[1]

# FILE_NAME = '../data/trace/TotalPkt.csv'
# FILE_TYPE = 'csv'
# OUT_FILE_NAME = '../data/csv/tcp1ms.csv'

# Load trace
raw_entries = []

In [44]:
import json

# Row format: key, length, timestamp, queue_len

flow_start_time = {}
flow_last_gap = {}
min_time = 10
max_time = 10000000

if FILE_TYPE == '.csv':
    with open(FILE_NAME, 'r') as f:
        f.readline()
        for line in f:
            key, len, time, queue_len = line.split(',')
            random_start_time = min_time + hash(key) % max_time
            if int(len) > 100: # remove tcp packets
                if key not in flow_start_time:
                    flow_start_time[key] = int(time)
                time_gap = int(time) - flow_start_time[key] 
                flow_last_gap
                # because there are iterations in ib_test
                # if time_gap > 1000000: # remote the gap according to last gap
                #     flow_start_time[key] = int(time)
                #     time_gap = 
                flow_last_gap
                time = random_start_time + time_gap
                raw_entries.append((int(key), int(len), int(time), int(queue_len)))

raw_entries.sort(key=lambda x: x[2])

if FILE_TYPE =='.json':
    total_packets = 0
    total_ecn_packets = 0  
    with open(FILE_NAME, 'r') as f:
        info = json.load(f)
        flows = info['flowStatistics']
        try:
            for f in flows:
                fid = f['flowId']
                packets = f['sentPkt']
                # judge if the packets is a list
                total_packets += len(f['sentPkt'])
                total_ecn_packets += len(f['recvEcn'])
                for p in packets:
                    raw_entries.append((fid, p['sizeByte'], p['timeNs'], 'nan'))
        except Exception as e:
            e.printStackTrace()
            print(f)
            exit(0)

    print(f"The ratio of ecn packets {1.0 * total_ecn_packets / total_packets}")
    

In [45]:
WINDOW_SIZE = 1000000  #  10 us


# 63470 5434942092 415599
# 57337 5436314330 426074
# 58247 5438192024 418870
# 64692 5440596512 424596
# 63792 5467074278 425821
# 60822 5468324916 426429
# 62545 5472902840 422887
# 59090 5483854398 426627
# 52426 5484376764 345394
# 58637 5511151342 427514


flow_data = {}
for flow, len, time, _ in raw_entries:
    if flow not in flow_data:
        flow_data[flow] = {}
    wid = int(time / WINDOW_SIZE)
    if wid not in flow_data[flow]:
        flow_data[flow][wid] = (0, 0)
    flow_data[flow][wid] = (flow_data[flow][wid][0] + 1, flow_data[flow][wid][1] + len)

result = {}
for flow, wid_dict in flow_data.items():
    result[flow] = sorted(wid_dict.items())

In [46]:
# write result to file, csv format :  flow_id, wid, pkt_cnt, byte_cnt

with open(OUT_FILE_NAME, 'w') as f:
    for flow, wid_dict in result.items():
        for wid, (pkt_cnt, byte_cnt) in wid_dict:
            f.write('%d,%d,%d,%d\n' % (flow, wid, pkt_cnt, byte_cnt))
            # f.write('%d %d %d\n' % (wid, pkt_cnt, byte_cnt))

with open(FILE_NAME+"_filter.csv", 'w') as f:
    for fid, len, time, _ in raw_entries:
        f.write('%d,%d,%d,%d\n' % (fid, len, time, 0))